In [1]:
import tensorflow as tf
import numpy as np
import time
from tensorflow.keras.layers import Layer
from tensorflow.keras import Model
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)
# autoencoder


class CompressionNet(Layer):
    def __init__(self, hidden_layer_size, input_size):
        # latent_layer_size: list of hidden layer size
        # input_size: int
        super(CompressionNet, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.latent_dim = hidden_layer_size[-1]
        self.input_size = input_size
        self.encoder = tf.keras.Sequential()
        self.encoder.add(tf.keras.Input(shape=(self.input_size,)))
        for size in self.hidden_layer_size[:-1]:
            self.encoder.add(tf.keras.layers.Dense(size, activation='tanh'))
        self.encoder.add(tf.keras.layers.Dense(self.latent_dim))
        self.decoder = tf.keras.Sequential()
        self.decoder.add(tf.keras.Input(shape=(self.latent_dim,)))
        for size in self.hidden_layer_size[::-1][1:]:
            self.decoder.add(tf.keras.layers.Dense(size, activation='tanh'))
        self.decoder.add(tf.keras.layers.Dense(self.input_size))

    def reconstruction_error_feature(self, x):
        # This is a part of estimation net input
        # In original paper, relative Euclidean distance and
        # cosine similarity is considered
        def euclid_norm(x):
            return tf.sqrt(tf.reduce_sum(tf.square(x), axis=1))
        hidden_state = self.encoder(x)
        x_new = self.decoder(hidden_state)
        # relative Euclidean distance
        loss_m = euclid_norm(x - x_new) / euclid_norm(x)
        # cosine similarity
        loss_c = tf.reduce_sum(x * x_new, axis=1) / \
            (euclid_norm(x) * euclid_norm(x_new))
        return tf.concat([loss_m[:, None], loss_c[:, None]], axis=1)

    def compute_loss(self, x):
        mse = tf.keras.losses.MeanSquaredError()
        hidden_state = self.encoder(x)
        x_new = self.decoder(hidden_state)
        loss = mse(x, x_new)
        return loss

    def call(self, inputs):
        hidden_state = self.encoder(inputs)
        reconstruction_error = self.reconstruction_error_feature(inputs)
        features = tf.concat([hidden_state, reconstruction_error], axis=1)
        return features


class EstimationNet(Layer):
    def __init__(self, hidden_layer_size, input_size):
        # latent_layer_size: list of hidden layer size
        # input_size: int
        super(EstimationNet, self).__init__()
        self.hidden_layer_size = hidden_layer_size
        self.input_size = input_size
        self.inference = tf.keras.Sequential()
        self.inference.add(tf.keras.Input(shape=(self.input_size,)))
        for size in self.hidden_layer_size[:-1]:
            self.inference.add(tf.keras.layers.Dense(size, activation='tanh'))
            self.inference.add(tf.keras.layers.Dropout(0.2))
        self.inference.add(tf.keras.layers.Dense(
            self.hidden_layer_size[-1], activation='softmax'))

    def call(self, inputs):
        outputs = self.inference(inputs)
        return outputs


class GMM:
    def __init__(self, z, gamma):
        # z : tf.Tensor, shape (n_samples, n_features)
        # gamma : tf.Tensor, shape (n_samples, n_component)
        gamma_sum = tf.reduce_sum(gamma, axis=0)  # shape (n_component, )
        self.phi = tf.reduce_mean(gamma, axis=0)  # shape (n_component, )
        # shape (n_component, n_features)
        self.mu = tf.matmul(gamma, z, transpose_a=True) / gamma_sum[:, None]
        z_centered = tf.sqrt(gamma[:, :, None]) * \
            (z[:, None, :] - self.mu[None, :, :])
        self.sigma = tf.einsum(
            'ikl,ikm->klm', z_centered, z_centered) / gamma_sum[:, None, None]
        # Calculate a cholesky decomposition of covariance in advance
        n_features = z.shape[1]
        min_vals = tf.linalg.diag(
            tf.ones(n_features, dtype=tf.float32)) * 1e-6
        self.L = tf.linalg.cholesky(self.sigma + min_vals[None, :, :])

    def compute_prob(self, z):
        # Instead of inverse covariance matrix, exploit cholesky decomposition
        # for stability of calculation.
        # shape (n_samples, n_component, n_features)
        z_centered = z[:, None, :] - self.mu[None, :, :]
        v = tf.linalg.triangular_solve(
            self.L, tf.transpose(z_centered, [1, 2, 0]))

        # log(det(Sigma)) = 2 * sum[log(diag(L))]
        log_det_sigma = 2.0 * \
            tf.reduce_sum(tf.math.log(tf.linalg.diag_part(self.L)), axis=1)

        # To calculate prob, use "log-sum-exp" (different from orginal paper)
        d = z.shape[1]
        logits = tf.math.log(self.phi[:, None]) - 0.5 * (tf.reduce_sum(
            tf.square(v), axis=1) + d * tf.math.log(2.0 * np.pi) +
            log_det_sigma[:, None])
        prob = - tf.reduce_logsumexp(logits, axis=0)
        return prob

    def energy_loss(self, z):
        return tf.reduce_mean(self.compute_prob(z))

    def diag_loss(self):
        diag_loss = tf.reduce_sum(
            tf.divide(1, tf.linalg.diag_part(self.sigma)))
        return diag_loss


class DAGMMnet(Model):
    def __init__(self, compress_hidden_layer,
                 estimate_hidden_layer, input_size):
        super(DAGMMnet, self).__init__()
        self.compress_hidden_layer = compress_hidden_layer
        self.latent_dim = compress_hidden_layer[-1]
        self.estimate_hidden_layer = estimate_hidden_layer
        self.input_size = input_size
        self.compress_net = CompressionNet(
            self.compress_hidden_layer, self.input_size)
        # +2 means reconstruct features
        self.estimate_net = EstimationNet(
            self.estimate_hidden_layer, self.latent_dim + 2)

    def compute_loss(self, x):
        lambda1 = 0.1
        lambda2 = 0.005
        z = self.compress_net(x)
        gamma = self.estimate_net(z)
        gmm_output = GMM(z, gamma)
        loss_1 = self.compress_net.compute_loss(x)
        loss_2 = gmm_output.energy_loss(z)
        loss_3 = gmm_output.diag_loss()
        return loss_1 + lambda1 * loss_2 + lambda2 * loss_3

    def call(self, x):
        z = self.compress_net(x)
        gamma = self.estimate_net(z)
        gmm_output = GMM(z, gamma)
        result = gmm_output.compute_prob(z)
        return result, z


@tf.function
def train_step(model, x, optimizer):
    with tf.GradientTape() as t:
        loss = model.compute_loss(x)
        gradient = t.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradient, model.trainable_variables))


def train(dataset, model, epochs, optimizer):
    for epoch in range(epochs):
        start = time.time()
        for data in dataset:
            train_step(model, data, optimizer)
        batch_num = 0
        total_loss = 0
        for data in dataset:
            batch_num += 1
            total_loss += model.compute_loss(data)
        print('Time for epoch {} is {} sec'.format(
            epoch + 1, time.time() - start))
        print('Loss: {}'.format(total_loss/batch_num))


In [2]:
#feature_extract = tf.keras.models.load_model('./feature_extract.h5')

In [2]:
train_x = np.load('./x.npy')

In [3]:
train_x = train_x.reshape(-1,784)

In [5]:
#train_x_tran = feature_extract(train_x)

In [4]:
my_model = DAGMMnet([10, 1], [10, 1], 784)

In [5]:
optimizer = tf.keras.optimizers.Adam()
data_set = tf.data.Dataset.from_tensor_slices(train_x).shuffle(2337).batch(128)
train(data_set, my_model, 30, optimizer)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

Time for epoch 1 is 4.094263553619385 sec
Loss: 5.540744781494141
Time for epoch 2 is 0.33045458793640137 sec
Loss: 3.5074245929718018
Time for epoch 3 is 0.3123800754547119 sec
Loss: 2.615748882293701
Time for epoch 4 is 0.2935361862182617 sec
Loss: 1.9973329305648804
Time for epoch 5 is 0.3161029815673828 sec
Loss: 1.7395750284194946
Time for epoch 6 is 0.2968461513519287 sec
Loss: 1.576899766921997
Time for epoch 7 is 0.31420183181762695 sec
Loss: 1.4559098482131958
Time for epoch 8 is 0.302744388580

In [6]:
result, _ = my_model(train_x)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



In [7]:
logit = np.exp(-result.numpy())

In [8]:
label = np.load('./y.npy')

In [9]:
label = (label == 1)

In [10]:
logit[label].mean()

3.8578598

In [11]:
logit[~label].mean()

4.0722003

In [12]:
logit.mean()

4.050097